<a href="https://colab.research.google.com/github/DongGwan0505/MNIST-classification-project/blob/main/Find_Best_Polynomial_Parameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys #시스템 관련 모듈 임포트
assert sys.version_info >= (3, 5) #파이썬 버전이 3.5이상인지 확인
import sklearn #scikit-learn이라는 머신러닝 라이브러리 임포트
assert sklearn.__version__ >= "0.20" #버전이 2.0이상인지 확인
import numpy as np #Numpy라이브러리 임포트 및 np로 부른다.
import os #운영체제 모듈 임포트

In [ ]:
PROJECT_ROOT_DIR = "." #PROJECT_ROOT_DIR 변수에 현재 디렉토리를 지정

DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")

#DATA_PATH 변수에는 PROJECT_ROOT_DIR과 "data"라는 폴더를 합친 경로를 할당
CHAPTER_ID   = "training_linear_models"

#"CHAPTER_ID 변수에는 training_linear_models"라는 문자열을 할당
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

#IMAGES_PATH 변수에 PROJECT_ROOT_DIR, "images" 폴더, CHAPTER_ID를 합친 경로를 할당
os.makedirs(IMAGES_PATH, exist_ok=True)

#IMAGES_PATH에 해당하는 디렉토리를 생성 exist_ok=True는 이미 디렉토리가 존재해도 오류를 발생시키지 않고 넘어간다
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [4]:
#그래프를 이미지 파일로 저장하는 함수. fig_id=그래프의 이름, 그리고, 해당 그래프를 이미지 파일로 저장. 저장 경로는 IMAGES_PATH에 fig_id와 확장자를 합친 것. tight_layout 매개변수가 True로 설정되어 있으면 그래프를 저장하기 전에 자동으로 레이아웃을 조정. 그래프는 지정된 확장자와 해상도로 저장
os.makedirs(DATA_PATH, exist_ok=True) #DATA_PATH에 해당하는 디렉토리를 생성
np.random.seed(25) #Numpy의 랜덤 시드 설정
%cd data #현재 작업 디렉토리를 "data" 폴더로 변경

X = np.load('X_datafile.npy')
y = np.load('y_datafile.npy') #저장된 NumPy 배열 데이터를 로드
X.shape #X는 300행 1열의 행렬이다.

[Errno 2] No such file or directory: 'data #현재 작업 디렉토리를 data 폴더로 변경'
/content


FileNotFoundError: [Errno 2] No such file or directory: 'X_datafile.npy'

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14) #축 레이블의 폰트 크기를 14로 설정
mpl.rc('xtick', labelsize=12) #x축 눈금 레이블의 폰트 크기를 12로 설정
mpl.rc('ytick', labelsize=12) #y축 눈금 레이블의 폰트 크기를 12로 설정

plt.plot(X, y, "b.")
plt.xlabel('X')
plt.ylabel('Y', rotation=0)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

#scikit-learn의 PolynomialFeatures를 사용하여 주어진 특성에 다항식 특성을 추가
poly_features = PolynomialFeatures (degree=2, include_bias=False)

#degree = 다항식의 차수
#include_bias=False는 절편(intercept)을 포함할 것인지 여부를 나타낸다.
X_poly = poly_features.fit_transform(X)
#Polynomial Features (𝑥,𝑥^2) 를 생성 한다

In [ ]:
from sklearn.linear_model import LinearRegression #scikit-learn에서 선형 회귀 모델을 임포트
lin_reg = LinearRegression() #선형 회귀 모델 객체를 생성
lin_reg.fit( X_poly, y) # y[i] =  f( X[i], X[i]*X[i] )
# 선형 회귀 모델을 주어진 다항 특성이 추가된 데이터 X_poly와 타겟 변수 y에 대해 학습시킨다.
lin_reg.intercept_, lin_reg.coef_ #학습된 모델의 절편(intercept),학습된 모델의 계수(coefficient) 값, 즉 특성들의 가중치(weights) 값을 반환
X_new = np.linspace(-2, 2, 300).reshape(300, 1)
#-2에서 2까지의 범위에서 300개의 데이터 포인트를 생성하고, 300x1의 형태로 재구성하여 X_new에 할당
X_new_poly = poly_features.transform( X_new )
#새로운 데이터에 대해 다항 특성을 변환하여 X_new_poly에 할당
y_new = lin_reg.predict(X_new_poly)
#변환된 새로운 데이터에 대해 학습된 모델로 예측을 수행하고, 예측값을 y_new에 할당

In [ ]:
plt.plot(X, y, "b.")
#원본 데이터인 X와 그에 해당하는 타겟 변수인 y를 파란색 점으로 시각화
plt.plot(X_new, y_new, "r-", linewidth=2, label="Predictions")
#새로운 데이터 X_new에 대한 예측값인 y_new를 빨간색 실선으로 시각화하고, 선의 두께를 2로 설정합니다. 또한, 이 예측 결과를 나타내는 라벨을 "Predictions"으로 지정

In [ ]:
#<B>
from sklearn.metrics import mean_squared_error #에러 값 평가
from sklearn.model_selection import train_test_split #데이터를 테스트, 트레이닝으로 나눈다.
def plot_learning_curves (model, X, y) :
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
#테스트 사이즈 20% 트레이닝 사이즈 80%
    train_errors, val_errors = [], []
    for m in range(1, len(X_train)) :
        model.fit(X_train[:m], y_train[:m])   # 1개, 2개, n개 instancece로 학습, ...
        y_train_predict = model.predict(X_train[:m])
        y_val_predict   = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))
    plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
    plt.legend(loc="upper right", fontsize=14)
    plt.xlabel("Training set size", fontsize=14)
    plt.ylabel("RMSE", fontsize=14)


In [ ]:
from sklearn.pipeline import Pipeline
polynomial_regression = Pipeline([
        ("poly_features", PolynomialFeatures(degree=1, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ]) #degree를 1~40까지 설정하여 n차 선형 회귀 모델 숫자 1을 1~40까지 설정한다.
plt.title("the 1st model") #1차 선형 회귀 모델 숫자 1을 1~40까지 설정한다.
plot_learning_curves(polynomial_regression, X, y)
plt.axis([0, 80, 0, 3])
plt.show()


In [ ]:
#<C>
np.random.seed(42) #같은 데이터를 이용해도 랜덤시드가 다르면 plot의 모습이 조금 달라진다.
X_train, X_val, y_train, y_val = train_test_split(X[:300], y[:300].ravel(), test_size=0.2, random_state=10)
#300개의 데이터를 모두 사용하여 테스트 사이즈를 20%로 설정한다.
from sklearn.linear_model import SGDRegressor
#Stochastic Gradient Descent로 다항 회귀 모델을 훈련
poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3, include_bias=False)),
        ("std_scaler", StandardScaler()) ])
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)
sgd_reg = SGDRegressor(max_iter=10, warm_start=True, #iteration을 10으로 설정했다.
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)
#iteration을 10으로 두었다.
from copy import deepcopy
minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(5000) :
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error      = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error :
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)
sgd_reg = SGDRegressor(max_iter=10, tol= 1e-3, warm_start=True, #iteration을 10으로 설정했다.
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)


In [ ]:
n_epochs = 5000 #epoch를 500으로 설정하였다.
train_errors, val_errors = [], []
for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))
best_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_epoch])
plt.annotate('Best model',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,)

best_val_rmse -= 0.03  # just to make the graph look better


In [ ]:
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation set")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="Training set")
plt.legend(loc="upper right", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("RMSE", fontsize=14)
plt.show()